In [12]:
import pandas as pd
import numpy as np
import spacy

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec

import morfeusz2

import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

# from sentence_transformers import SentenceTransformer
import umap
import hdbscan

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
from sklearn.model_selection import train_test_split
from sklearn import utils

In [14]:
# nlp_core = spacy.load("pl_core_news_lg") # nlp
nlp_pl = spacy.load('pl_spacy_model') # nlp37

In [15]:
# model = SentenceTransformer('dkleczek/bert-base-polish-uncased-v1')

## Load data

In [16]:
# https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e

In [19]:
df_dem = pd.read_csv('../datasets/scrapped/demagog_features.csv', sep=';')
df_dem.head()

,assestment,text,text_clean,sentiment_all,sentiment_avg,uniq_words,uniq_lemm,err,net,ADJ,ADV,NOUN
0,Nieweryfikowalne,"Generalnie, jak pokazują dane i szacunki, to n...","Generalnie, jak pokazują dane i szacunki, to ...",-0.008995,-0.083364,33.0,29.0,3.0,1.0,0.212121,0.060606,0.212121
1,Prawda,Według ich (ukraińskich – przyp. Demagog) dany...,"Według ich ukraińskich danych w Polsce, czy n...",-0.008995,-0.008995,18.0,17.0,0.0,3.0,0.055556,0.055556,0.166667
2,Fałsz,"Po pierwsze, jest 51,25 proc. ludzi tylko zasz...","Po pierwsze, jest 51,25 proc. ludzi tylko zasz...",-0.008995,-0.045055,24.0,24.0,0.0,1.0,0.125000,0.125000,0.083333
3,Prawda,"Po pierwsze, system bankowy w Polsce, no, ma s...","Po pierwsze, system bankowy w Polsce, no, ma s...",0.026141,-0.081037,19.0,19.0,1.0,1.0,0.157895,0.000000,0.263158
4,Fałsz,"Magazyny gazu mamy pełne tylko w 60%, bo w lis...","Magazyny gazu mamy pełne tylko w 60%, bo w lis...",-0.024012,-0.024012,23.0,23.0,2.0,2.0,0.086957,0.000000,0.173913


In [20]:
df_oko = pd.read_csv('../datasets/oko.press/okopress_features.csv', sep=';')
df_oko.head()

,sub_napis,sub_stan_zegara,text_clean,sentiment_all,sentiment_avg,uniq_words,uniq_lemm,err,net,ADJ,ADV,NOUN
0,Rządy Tuska to również doprowadzenie do wyzysk...,falsz,Rządy Tuska to również doprowadzenie do wyz...,-0.008995,-0.195652,28.0,27.0,4.0,5.0,0.071429,0.071429,0.285714
1,Absurdy i marnotrawstwo pokazuje najlepiej pro...,blisko_prawdy,Absurdy i marnotrawstwo pokazuje najlepiej pro...,-0.188746,-0.188746,26.0,25.0,2.0,1.0,0.038462,0.038462,0.423077
2,Przez 15 lat finansowaliście budowę korwety Ga...,blisko_prawdy,Przez 15 lat finansowaliście budowę korwety ...,-0.201911,-0.159423,35.0,35.0,3.0,3.0,0.085714,0.028571,0.371429
3,Dzisiaj ponad 65 procent długu państwowego jes...,falsz,Dzisiaj ponad 65 procent długu państwowego je...,-0.008995,-0.008995,23.0,23.0,2.0,0.0,0.173913,0.043478,0.260870
4,"Polska jest gotowa przyjąć każdego uchodźcę, k...",falsz,Polska jest gotowa przyjąć każdego uchodźc...,-0.007072,-0.007072,34.0,34.0,4.0,3.0,0.147059,0.058824,0.235294


In [21]:
df_oko.columns

Index(['sub_napis', 'sub_stan_zegara', 'text_clean', 'sentiment_all',
       'sentiment_avg', 'uniq_words', 'uniq_lemm', 'err', 'net', 'ADJ', 'ADV',
       'NOUN'],
      dtype='object')

In [33]:
df = pd.concat(
    [
        df_dem[['assestment', 'text_clean', 'sentiment_all', 'sentiment_avg', 'uniq_words', 'uniq_lemm', 'err', 'net', 'ADJ', 'ADV','NOUN']],
        (df_oko[['sub_stan_zegara', 'text_clean', 'sentiment_all', 'sentiment_avg', 'uniq_words', 'uniq_lemm', 'err', 'net', 'ADJ', 'ADV','NOUN']]
         .rename(columns={'sub_stan_zegara':'assestment'}))
    ],
    ignore_index = True
)

## Create tokenizer

In [34]:
# stopwords = nlp_core.Defaults.stop_words
stopwords = nlp_pl.Defaults.stop_words

In [35]:
def tokenize(txt):
    txt = txt.replace('\n', ' ')
    # doc = nlp_core(txt)
    doc = nlp_pl(txt)
    
    words = [
        token.lemma_.lower()
        for token in doc 
        if 
            not token.is_stop 
            and not token.is_punct 
            and not token.is_stop 
            and token.text != ' '
            and token.lemma_ not in stopwords
            and len(token.text) > 1 ]
    
    return words

In [36]:
df['tokens'] = df['text_clean'].progress_apply(lambda x: tokenize(x))

100%|███████████████████████████████████████| 7760/7760 [08:44<00:00, 14.81it/s]


In [42]:
df['text_clean'].values[444]

'Czyli tak:'

In [37]:
df['tokens_str'] = df['tokens'].progress_apply(lambda x: ' '.join(x))

100%|███████████████████████████████████| 7760/7760 [00:00<00:00, 293070.28it/s]


In [38]:
df.columns

Index(['assestment', 'text_clean', 'sentiment_all', 'sentiment_avg',
       'uniq_words', 'uniq_lemm', 'err', 'net', 'ADJ', 'ADV', 'NOUN', 'tokens',
       'tokens_str'],
      dtype='object')

In [43]:
df[
    ['assestment', 'text_clean', 
     'sentiment_all', 'sentiment_avg', 'uniq_words', 'uniq_lemm', 'err', 'net', 
     'ADJ', 'ADV', 'NOUN', 'tokens_str']
].to_csv('../datasets/ready2use/fake_news_features_tokens.csv', sep=';', index=False)

In [32]:
df['tokens'].str.join('|').str.get_dummies().max().max()

1

## Create embeddings - BERT

In [ ]:
# https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6

In [ ]:
# embeddings = model.encode(df['text_clean'].values, show_progress_bar=True)

In [ ]:
# umap_embeddings = umap.UMAP(n_neighbors=15, 
#                             n_components=5, 
#                             metric='cosine').fit_transform(embeddings)

In [ ]:
# df[['e0', 'e1', 'e2', 'e3', 'e4']] = 0

# df[['e0', 'e1', 'e2', 'e3', 'e4']] = umap_embeddings

# df_learn = df[
#     ['assestment', 'text_clean', 
#      'sentiment_all', 'sentiment_avg',
#      'uniq_words', 'uniq_lemm', 'err', 'net', 'ADJ', 'ADV', 'NOUN',
#      'e0', 'e1', 'e2', 'e3', 'e4']
# ]

# df_learn.to_csv('../datasets/scrapped/demagog_features_emb_umap.csv', sep=';', index=False)

In [ ]:
# cluster = hdbscan.HDBSCAN(min_cluster_size=15,
#                           metric='euclidean',                      
#                           cluster_selection_method='eom').fit(umap_embeddings)

In [ ]:
# # Prepare data
# umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# result = pd.DataFrame(umap_data, columns=['x', 'y'])
# result['labels'] = cluster.labels_

# # Visualize clusters
# fig, ax = plt.subplots(figsize=(20, 10))
# outliers = result.loc[result.labels == -1, :]
# clustered = result.loc[result.labels != -1, :]
# plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=5)
# plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=5, cmap='hsv_r')
# plt.colorbar()

In [ ]:
# docs_df = pd.DataFrame(df['text_clean'].values, columns=["Doc"])
# docs_df['Topic'] = cluster.labels_
# docs_df['Doc_ID'] = range(len(docs_df))
# docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

In [ ]:
def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words=stopwords).fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(df['text_clean'].values))

In [ ]:
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

# top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
# topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

## Create documents

In [ ]:
documents = [TaggedDocument(doc, [str(i)]) for i, doc in enumerate(df['tokens'].values)]

In [ ]:
# test/train split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    documents, df['assestment'].values, test_size=0.33, random_state=0)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
len(documents)

## Distributed Bag of Words (DBOW)

In [ ]:
# https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4

In [ ]:
vec_size = 5

In [39]:
model_dbow = Doc2Vec(dm=0, vector_size=vec_size, negative=5, hs=0, min_count=2, sample = 0, workers=-1)
model_dbow.build_vocab([x for x in tqdm(X_train)])

100%|██████████████████████████████████| 3276/3276 [00:00<00:00, 1327331.91it/s]


In [40]:
model_dbow.dv.vectors.shape

(3276, 5)

In [41]:
for epoch in tqdm(range(30)):
    model_dbow.train(utils.shuffle([x for x in X_train]), total_examples=len(X_train), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████████████████████████████████████| 30/30 [00:00<00:00, 167.88it/s]


In [42]:
model_dbow.dv.vectors.shape

(3276, 5)

In [43]:
model_dbow.infer_vector(X_train[0][0], epochs=20)

array([ 0.04901308,  0.02188243, -0.06371115,  0.073976  ,  0.08626302],
      dtype=float32)

## Distributed Memory (DM)

In [44]:
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=vec_size, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(X_train)])

100%|██████████████████████████████████| 3276/3276 [00:00<00:00, 2455860.57it/s]


In [45]:
for epoch in tqdm(range(30)):
    model_dmm.train(utils.shuffle([x for x in X_train]), total_examples=len(X_train), epochs=1)
    model_dmm.alpha -= 0.002
    model_dmm.min_alpha = model_dmm.alpha

100%|███████████████████████████████████████████| 30/30 [00:33<00:00,  1.11s/it]


In [46]:
model_dmm.infer_vector(X_train[0][0], epochs=20)

array([0.03888087, 0.11844292, 0.09204848, 0.09457888, 0.04699333],
      dtype=float32)

## Model Pairing

In [47]:
new_model = ConcatenatedDoc2Vec([model_dbow, model_dmm])

In [48]:
#model.train(X_train, total_examples=model.corpus_count, epochs=model.epochs)

In [49]:
model = model_dbow

doc_id = 36
# inferred_vector = model_loaded.infer_vector(['酒精', '用', '啥', '稀释'])
inferred_vector = model.infer_vector(X_train[doc_id].words)
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

print(X_train[36])

TaggedDocument<['eurostat', 'umieścił', 'polskę', 'drugi', 'miejsce', 'ue', 'jeśli', 'chodzić', 'przyjmowanie', 'uchodźców', 'paść', 'liczba', '400', 'tys.', 'osób', 'oczywiście', 'są', 'lud', 'ukraina'], ['2525']>


In [58]:
# model_dbow.wv.most_similar('covid')

## Apply embeddings

In [59]:
df[[ 'e'+str(i) for i in range(vec_size*2) ]] = [new_model.infer_vector(d[0]).tolist() for d in documents]

In [60]:
df_learn = df[
    ['assestment', 'text_clean', 
     'sentiment_all', 'sentiment_avg',
     'uniq_words', 'uniq_lemm', 'err', 'net', 'ADJ', 'ADV', 'NOUN',
     'e0', 'e1', 'e2', 'e3', 'e4',
     'e5', 'e6', 'e7', 'e8', 'e9']
]

In [61]:
df_learn.to_csv('../datasets/scrapped/demagog_features_emb_pl.csv', sep=';', index=False)